In [178]:
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score

In [179]:
train_data=pd.read_csv('/home/dugrarao/Documents/NLP/Tweetproject/train_2kmZucJ.csv')


In [180]:
#cleaning train data
for i in range(len(train_data['tweet'])):
    txt = train_data.loc[i]["tweet"]
    txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace @user tags
    txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
    txt=re.sub("[^a-zA-Z]", " ",txt)#replace hashtags
    train_data.at[i,"tweet"]=txt


In [181]:
#removing words that are less than 3
train_data['tweet']=train_data['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


In [182]:
#Tokenizing
tokenized_train_data=train_data['tweet'].apply(lambda x: x.split())


In [183]:
train_data['tweet'] = tokenized_train_data.apply(lambda x:' '.join([lem.lemmatize(i) for i in x])) # lemmatizing


In [184]:
#bagofwords
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer_train_data = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow_train_data = bow_vectorizer_train_data.fit_transform(train_data['tweet'])
bow_train_data

<7920x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 49661 stored elements in Compressed Sparse Row format>

In [185]:
#Tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer_train_data = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf_train_data = tfidf_vectorizer_train_data.fit_transform(train_data['tweet'])
tfidf_train_data

<7920x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 49661 stored elements in Compressed Sparse Row format>

In [186]:
#splitting train data
x_train, x_test, y_train, y_test = train_test_split(tfidf_train_data,train_data["label"], test_size = 0.2, random_state = 42)
x_train,x_test

(<6336x1000 sparse matrix of type '<class 'numpy.float64'>'
 	with 39734 stored elements in Compressed Sparse Row format>,
 <1584x1000 sparse matrix of type '<class 'numpy.float64'>'
 	with 9927 stored elements in Compressed Sparse Row format>)

In [187]:
#Building Logistic Regression model
lreg = LogisticRegression()
lreg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [188]:
prediction_int1=lreg.predict_proba(x_test)
train_data_predicted_val=prediction_int1[:,1] >= 0.4
train_data_predicted_val=train_data_predicted_val.astype(np.int)

In [189]:
f1_score(y_test,train_data_predicted_val)

0.7929824561403509

In [190]:
confusion_matrix(y_test,train_data_predicted_val)

array([[1068,   84],
       [  93,  339]])

In [191]:
accuracy_score(y_test,train_data_predicted_val)

0.8882575757575758

In [192]:
#For test data
test_data=pd.read_csv('test_oJQbWVk.csv')


In [193]:
#Cleaning test data
for i in range(len(test_data['tweet'])):
    txt = test_data.loc[i]["tweet"]
    txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace username-tags
    txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
    txt=re.sub("[^a-zA-Z]", " ",txt)#replace hashtags
    test_data.at[i,"tweet"]=txt
#removing words that are less than 3
test_data['tweet']=test_data['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#Tokenizing
tokenized_test_data=test_data['tweet'].apply(lambda x: x.split())
test_data['tweet'] = tokenized_test_data.apply(lambda x:' '.join([lem.lemmatize(i) for i in x])) # lemmatizing


In [194]:
#Bagofwords
bow_vectorizer_test_data = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow_test_data = bow_vectorizer_test_data.fit_transform(test_data['tweet'])


In [195]:
#Tf-idf

tfidf_vectorizer_test_data = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf_test_data = tfidf_vectorizer_test_data.fit_transform(test_data['tweet'])
tfidf_test_data

<1953x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 12191 stored elements in Compressed Sparse Row format>

In [196]:
lreg = LogisticRegression()
lreg.fit(tfidf_train_data, train_data['label'])
prediction_int1=lreg.predict_proba(tfidf_test_data)
test_data_predicted_val=prediction_int1[:,1] >= 0.4
test_data_predicted_val=test_data_predicted_val.astype(np.int)

In [197]:
#Predictions to csv
final_result = pd.DataFrame({'id':test_data['id'],'label':test_data_predicted_val})
final_result.to_csv('LogisticRegressionOutput.csv',index=False)

